In [75]:
%matplotlib inline
######################################################################################################
#
#         SCRIPT DESCRIPTION: Basic Q-Learning in a single intersection
#
#         Input: Map name, route file
#         Output: Tripinfo.xml
#
#####################################################################################################

# Set OPERATING SYSTEM
OS = "iOS"
# Set MAP and time
used_map = 'triple_straight_random'
simulation_length = 900000
demand = "fixed"
# Select whether to use previous experience
reloadQm  = True
saveQm    = True
# Select whether to color code the vehicles
color_vehicles = True
persistent_color = False

# Set Path for Sumo Home folder
if OS == "iOS":
    sumo_home_dir = "/Users/acabrejasegea/sumo-0.32/"
elif OS == "windows":
    sumo_home_dir = "C:\Program Files (x86)\DLR\Sumo"
else:
    sys.exit("Please declare a supported operative system")

# Import basic libraries
import os
import sys
import optparse
import subprocess
import random
import random
import pdb
import xmltodict
import re
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pickle
from __future__ import absolute_import
from __future__ import print_function

# Current working directory
working_dir = os.getcwd()

# Set environment variable SUMO_HOME
try:
    # Use this line only to manually define SUMO_HOME path
    os.environ["SUMO_HOME"] = sumo_home_dir
    sys.path.append(os.path.join(os.environ['SUMO_HOME'], 'tools'))
except ImportError:
    sys.exit(
        "Please declare environment variable 'SUMO_HOME' as the root directory of your sumo installation (it should contain folders 'bin', 'tools' and 'docs')")

# Import API libraries
from sumolib import checkBinary
import traci

In [2]:
%%bash
echo $SUMO_HOME

/Users/acabrejasegea/sumo-0.32/


In [3]:
%%javascript
IPython.notebook.kernel.execute('notebook_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [4]:
# Set important directories
if OS=="iOS":
    networks_dir = "/Users/acabrejasegea/sumo-0.32/Data/Networks/"
    scripts_dir = "/Users/acabrejasegea/sumo-0.32/tools/"
    runners_dir = "/Users/acabrejasegea/Desktop/ATI_Project/MLforFlowOptimisation/"
elif OS=="windows":
    networks_dir = "C:\\Program Files (x86)\\DLR\\Sumo\\data\\Networks\\"
    scripts_dir = "C:\\Program Files (x86)\\DLR\\Sumo\\tools\\"
    runners_dir = "C:\\Users\\CabrejasEgeaAlvaro\\Desktop\\ATI\\05_ML_Traffic_Lights\\MLforFlowOptimisation\\"
else:
    sys.exit("Please declare a supported operative system")
notebook = (os.path.splitext(notebook_name))[0]

In [5]:
def keywithmaxval(d):                   #Function used in Example control algorithm
     """ a) create a list of the dict's keys and values; 
         b) return the key with the max value"""  
     v=list(d.values())
     k=list(d.keys())
     return k[v.index(max(v))]

def getEdgeData(edgeIDs):
    #Edge data order (type: double / integer): [CO2, CO, Hydrocarbons, NOx, PMx, Noise,
    #mean Vehicle Length, mean Speed,occupancy, number of vehicles (integer),
    #estimated travel time, Waiting time]
    #http://sumo.dlr.de/daily/pydoc/traci._edge.html 
    data = []

    for i in range(0,len(edgeIDs)):
        data.append(traci.edge.getCO2Emission(edgeIDs[i]))
        data.append(traci.edge.getCOEmission(edgeIDs[i]))
        data.append(traci.edge.getHCEmission(edgeIDs[i]))
        data.append(traci.edge.getNOxEmission(edgeIDs[i]))
        data.append(traci.edge.getPMxEmission(edgeIDs[i]))
        data.append(traci.edge.getNoiseEmission(edgeIDs[i]))
        data.append(traci.edge.getLastStepLength(edgeIDs[i]))
        data.append(traci.edge.getLastStepMeanSpeed(edgeIDs[i]))
        data.append(traci.edge.getLastStepOccupancy(edgeIDs[i]))
        data.append(traci.edge.getLastStepVehicleNumber(edgeIDs[i]))
        data.append(traci.edge.getTraveltime(edgeIDs[i]))
        data.append(traci.edge.getWaitingTime(edgeIDs[i]))
    return data

def getInductionLoopData(inductionLoopIDs):
    #Inductive Loop data order (type: double / integer): [mean Vehicle Length,
    #mean Speed, occupancy, number of vehicles (integer)]
    #http://sumo.dlr.de/daily/pydoc/traci._inductionloop.html 
    data = []

    for i in range(0,len(inductionLoopIDs)):
        data.append(traci.inductionloop.getLastStepLength(inductionLoopIDs[i]))
        data.append(traci.inductionloop.getLastStepMeanSpeed(inductionLoopIDs[i]))
        data.append(traci.inductionloop.getLastStepOccupancy(inductionLoopIDs[i]))
        data.append(traci.inductionloop.getLastStepVehicleNumber(inductionLoopIDs[i]))
    return data

def getLaneData(laneIDs):
    #Lane data order (type: double / integer): [CO2, CO, Hydrocarbons, NOx, PMx, Noise,
    #mean Vehicle Length, mean Speed, occupancy, number of vehicles (integer),
    #estimated travel time, Waiting time]
    #http://sumo.dlr.de/daily/pydoc/traci._lane.html 
    data = []

    for i in range(0,len(laneIDs)):
        data.append(traci.lane.getCO2Emission(laneIDs[i]))
        data.append(traci.lane.getCOEmission(laneIDs[i]))
        data.append(traci.lane.getHCEmission(laneIDs[i]))
        data.append(traci.lane.getNOxEmission(laneIDs[i]))
        data.append(traci.lane.getPMxEmission(laneIDs[i]))
        data.append(traci.lane.getNoiseEmission(laneIDs[i]))
        data.append(traci.lane.getLastStepLength(laneIDs[i]))
        data.append(traci.lane.getLastStepMeanSpeed(laneIDs[i]))
        data.append(traci.lane.getLastStepOccupancy(laneIDs[i]))
        data.append(traci.lane.getLastStepVehicleNumber(laneIDs[i]))
        data.append(traci.lane.getTraveltime(laneIDs[i]))
        data.append(traci.lane.getWaitingTime(laneIDs[i]))
    return data

def getLaneAreaData(laneAreaIDs):
    #Lane area characteristic order (type: double / integer): [jam length (m), jam length (no. vehicles),
    #mean Speed, occupancy, number of vehicles (integer)]
    #http://sumo.dlr.de/daily/pydoc/traci._lanearea.html 
    data = []
    
    for i in range(0,len(laneAreaIDs)):
        data.append(traci.lanearea.getJamLengthMeters(laneAreaIDs[i]))
        data.append(traci.lanearea.getJamLengthVehicle(laneAreaIDs[i]))
        data.append(traci.lanearea.getLastStepMeanSpeed(laneAreaIDs[i]))
        data.append(traci.lanearea.getLastStepOccupancy(laneAreaIDs[i]))
        data.append(traci.lanearea.getLastStepVehicleNumber(laneAreaIDs[i]))
    return data

def getMultiEntryExitData(multiEntryExitIDs):
    #Multi entry exit detection area data order (type: double / integer):
    #mean Speed, number of vehicles (integer)]
    #http://sumo.dlr.de/daily/pydoc/traci._multientryexit.html 
    data = []

    for i in range(0,len(multiEntryExitIDs)):
        data.append(traci.multientryexit.getLastStepMeanSpeed(multiEntryExitIDs[i]))
        data.append(traci.multientryexit.getLastStepVehicleNumber(multiEntryExitIDs[i]))
    return data

def getSimulationData():
    #Simulation data order (type: double):
    #[current time, number of vehicles entering the network]
    #http://sumo.dlr.de/daily/pydoc/traci._simulation.html
    data = []

    data.append(traci.simulation.getCurrentTime())
    data.append(traci.simulation.getDepartedNumber())
    return data

def getTrafficLightData(trafficLightIDs):
    #Traffic light data = [current phase ID, phase duration,
    #current traffic phase]
    #http://sumo.dlr.de/daily/pydoc/traci._trafficlights.html 
    data = []

    for i in range(0,len(trafficLightIDs)):
        data.append(traci.trafficlights.getPhase(trafficLightIDs[i]))
        data.append(traci.trafficlights.getPhaseDuration(trafficLightIDs[i])) #this will reference the fixed time
        data.append(traci.trafficlights.getRedYellowGreenState(trafficLightIDs[i]))
    return data

def getVehicleData(vehicleIDs):
    #Individual vehicle data = [CO2, CO, Hydrocarbons, NOx, PMx, Noise,
    #emission class, total distance travelled (odometer), length,
    #list of upcoming traffic lights [(tlsID, tlsIndex, distance, state), ...], position (m,m),
    #vehicle route (edge IDs),current Speed, Vehicle Class, Waiting Time (below 0.1m/s)]
    #http://sumo.dlr.de/daily/pydoc/traci._vehicle.html 
    data = []

    for i in range(0,len(vehicleIDs)):
        data.append(traci.vehicle.getCO2Emission(vehicleIDs[i]))
        data.append(traci.vehicle.getCOEmission(vehicleIDs[i]))
        data.append(traci.vehicle.getHCEmission(vehicleIDs[i]))
        data.append(traci.vehicle.getNOxEmission(vehicleIDs[i]))
        data.append(traci.vehicle.getPMxEmission(vehicleIDs[i]))
        data.append(traci.vehicle.getNoiseEmission(vehicleIDs[i]))
        data.append(traci.vehicle.getEmissionClass(vehicleIDs[i]))
        data.append(traci.vehicle.getDistance(vehicleIDs[i]))
        data.append(traci.vehicle.getLength(vehicleIDs[i]))
        data.append(traci.vehicle.getNextTLS(vehicleIDs[i]))
        data.append(traci.vehicle.getPosition(vehicleIDs[i]))
        data.append(traci.vehicle.getRoute(vehicleIDs[i]))
        data.append(traci.vehicle.getSpeed(vehicleIDs[i]))
        data.append(traci.vehicle.getVehicleClass(vehicleIDs[i]))
        data.append(traci.vehicle.getWaitingTime(vehicleIDs[i]))
    return data

def getStaticLaneData(laneIDs):
    #Static lane data = [Lane Length, Max Speed]
    #http://sumo.dlr.de/daily/pydoc/traci._lane.html
    data = []

    #Get lane characteristics
    for i in range(0,len(laneIDs)):
        data.append(traci.lane.getLength(laneIDs[i]))
        data.append(traci.lane.getMaxSpeed(laneIDs[i]))
    return data

def getStaticTrafficLightData(trafficLightIDs):
    #Static traffic light data = [lanes controlled by traffic lights, links controlled
    #by traffic lights, ]
    #http://sumo.dlr.de/daily/pydoc/traci._trafficlights.html 
    data = []

    for i in range(0,len(trafficLightIDs)):
        data.append(traci.trafficlight.getControlledLanes(trafficLightIDs[i]))
        data.append(traci.trafficlight.getControlledLinks(trafficLightIDs[i]))
    return data

In [6]:
# Creates a route file for the simulation
def generate_routefile(simulation_length):
    random.seed(42)  # make tests reproducible
    time_counter = 0
    # demand per second from different directions
    if demand == "fixed":
        pWE  = 1. / 6
        pEW  = 1. / 6
        pNS1 = 1. / 20
        pNS2 = 1. / 20
        pNS3 = 1. / 20
        pSN1 = 1. / 20
        pSN2 = 1. / 20
        pSN3 = 1. / 20
    elif demand == "random":
        thresholds = np.linspace(0,simulation_length,5)
        pWE_v  = np.linspace(0.04, 0.25, 5)
        pEW_v  = np.linspace(0.04, 0.25, 5)
        pNS1_v = np.linspace(0.02, 0.10, 5)
        pNS2_v = np.linspace(0.02, 0.10, 5)
        pNS3_v = np.linspace(0.02, 0.10, 5)
        pSN1_v = np.linspace(0.02, 0.10, 5)
        pSN2_v = np.linspace(0.02, 0.10, 5)
        pSN3_v = np.linspace(0.02, 0.10, 5)
    else:
        sys.exit("Error in kind of demand, please choose random or fixed")
    
    
    # Algorithmic access to routefile (names need to be consistent)
    with open(os.path.join(networks_dir, used_map, (used_map + '.rou.xml')), "w") as routes:
        print("""<routes>
        <vType id="typeWE" accel="0.8" decel="4.5" sigma="0.5" length="5" minGap="2" maxSpeed="25" guiShape="passenger"/>
        <vType id="typeNS" accel="0.8" decel="4.5" sigma="0.5" length="5" minGap="2" maxSpeed="25" guiShape="passenger"/>

        <route id="right" edges="iW iCW oCE oE" />
        <route id="left" edges="iE iCE oCW oW" />
        <route id="down1" edges="iN1 oS1" />
        <route id="down2" edges="iN2 oS2" />
        <route id="down3" edges="iN3 oS3" />
        <route id="up1" edges="iS1 oN1" />
        <route id="up2" edges="iS2 oN2" />
        <route id="up3" edges="iS3 oN3" />""", file=routes)
        lastVeh = 0
        vehNr = 0
        for i in range(simulation_length):
            if demand == "random":
                if i in thresholds:
                    pWE  = random.choice(pWE_v)
                    pEW  = random.choice(pEW_v)
                    pNS1 = random.choice(pNS1_v)
                    pNS2 = random.choice(pNS2_v)
                    pNS3 = random.choice(pNS3_v)
                    pSN1 = random.choice(pSN1_v)
                    pSN2 = random.choice(pSN2_v)
                    pSN3 = random.choice(pSN3_v)
            
            if random.uniform(0, 1) < pWE:
                print('    <vehicle id="right_%i" type="typeWE" route="right" depart="%i" color="0,1,0"/>' % (
                    vehNr, i), file=routes)
                vehNr += 1
                lastVeh = i
            if random.uniform(0, 1) < pEW:
                print('    <vehicle id="left_%i" type="typeWE" route="left" depart="%i" color="0,1,0"/>' % (
                    vehNr, i), file=routes)
                vehNr += 1
                lastVeh = i
            if random.uniform(0, 1) < pNS1:
                print('    <vehicle id="down_%i" type="typeNS" route="down1" depart="%i" color="0,1,0"/>' % (
                    vehNr, i), file=routes)
                vehNr += 1
                lastVeh = i
            if random.uniform(0, 1) < pNS2:
                print('    <vehicle id="down_%i" type="typeNS" route="down2" depart="%i" color="0,1,0"/>' % (
                    vehNr, i), file=routes)
                vehNr += 1
                lastVeh = i
            if random.uniform(0, 1) < pNS3:
                print('    <vehicle id="down_%i" type="typeNS" route="down3" depart="%i" color="0,1,0"/>' % (
                    vehNr, i), file=routes)
                vehNr += 1
                lastVeh = i
            if random.uniform(0, 1) < pSN1:
                print('    <vehicle id="down_%i" type="typeNS" route="up1" depart="%i" color="0,1,0"/>' % (
                    vehNr, i), file=routes)
                vehNr += 1
                lastVeh = i
            if random.uniform(0, 1) < pSN2:
                print('    <vehicle id="down_%i" type="typeNS" route="up2" depart="%i" color="0,1,0"/>' % (
                    vehNr, i), file=routes)
                vehNr += 1
                lastVeh = i
            if random.uniform(0, 1) < pSN3:
                print('    <vehicle id="down_%i" type="typeNS" route="up3" depart="%i" color="0,1,0"/>' % (
                    vehNr, i), file=routes)
                vehNr += 1
                lastVeh = i
        print("</routes>", file=routes)

In [7]:
class NetworkParser:

    def __init__(self, file_path):
        self.junctions = dict()
        self.junction_lights = dict()

        with open(file_path) as f:
            self.file = xmltodict.parse(f.read())

    def get_lane_ids(self):
        for junction in self.file['net']['junction']:
            if junction['@type'] == 'traffic_light':
                self.junctions[junction['@id']] = junction['@incLanes'].split(' ')
        return self.junctions

    # Given a LaneID it returns the traffic light ID
    def get_trafficlights_by_laneID(self, laneID):
        for connection in self.file['net']['connection']:
            if connection['@from'] == laneID.split('_')[0] and '@tl' in connection:
                return connection['@tl']

    def get_junction_trafficlights(self):
        # If the dictionarty for junctions is empty
        if not self.junctions:
            self.get_lane_ids()
        for junctionID in self.junctions.keys():
            for lane in self.junctions[junctionID]:
                traffic_light = self.get_trafficlights_by_laneID(lane)
                if traffic_light in self.junction_lights and lane not in self.junction_lights[traffic_light]:
                    self.junction_lights[traffic_light].append(lane)
                else:
                    self.junction_lights[traffic_light] = []
        return self.junction_lights

    def get_traffic_light(self, laneID):
        if not self.junction_lights:
            self.get_junction_trafficlights()
        for traffic_light, lanes in self.junction_lights.iteritems():
            if laneID in lanes:
                return traffic_light

    def get_phases(self, traffic_light):
        phases = []
        z = 0
        for tls in self.file['net']['tlLogic']:
            if z < 1:
                a=0
            z+=1
            if tls['@id'] == traffic_light:
                for tls_phase in tls['phase']:
                    phases.append(tls_phase)
        return phases

In [76]:
class Qlearner:
    def __init__(self,TLid,lanes,phases):
        self.jid     = TLid
        self.laneids = lanes
        self.Nlanes  = len(self.laneids)                                #len(nodes.junctions[self.jid])
        self.phases  = range(0, 23)                            #traci.trafficlights.getallPhases
        num_phases   = len(self.phases)
        self.Qm      = np.zeros((275, 11))
        
        self.state   = 0
        self.action  = 0
        self.neighbor_state = 0
                                
        self.cycle_time = 90
        
        self.historic_reward = list()
  
    def set_action(self,action):                               
        traci.trafficlights.setProgram(self.jid, str(action))
                                                                

    def get_reward(self):
        # Reward defined as a measure of the amount of waiting in the intersection
        return -sum([traci.lane.getLastStepHaltingNumber(lane_id) for lane_id in self.laneids])
 

    def get_state(self,threshold_l=3):
        # State is defined as a combination b/w 5 load level for the 2 routes
        NS_occupancy = traci.lane.getLastStepHaltingNumber(self.laneids[0]) + traci.lane.getLastStepHaltingNumber(self.laneids[2])
        EW_occupancy = traci.lane.getLastStepHaltingNumber(self.laneids[1]) + traci.lane.getLastStepHaltingNumber(self.laneids[3])

        NS_state = np.trunc(NS_occupancy/3)
        EW_state = np.trunc(EW_occupancy/3)
        
        if NS_state > 4:
            NS_state = 4
        if EW_state >4:
            EW_state = 4
        
        if self.jid == learners[0].jid:
            self.neighbor_state = 0
        elif self.jid == learners[1].jid:
            self.neighbor_state = traci.trafficlights.getProgram("Centre1")
        elif self.jid == learners[2].jid:
            self.neighbor_state = traci.trafficlights.getProgram("Centre2")
        else:
            sys.exit("Error in getProgram or self.jid")
        
        state = int(NS_state + 5* EW_state + 25* np.float64(self.neighbor_state))

        return(state)

    def update(self,gamma=0.8,alpha=0.95,epsilon=0.0):
        
        # Get current state, action from class
        state    = self.state
        action   = self.action
        # Generate new action and the reward
        newstate = self.get_state()
        reward   = self.get_reward()
        self.historic_reward.append(reward)

        # Update the Q matrix
        self.Qm[state, action] = self.Qm[state, action]*(1-alpha) + alpha*(reward + gamma*np.max(self.Qm[newstate,:]))

        # Explore or exploit
        r=np.random.rand(1)
        if r > epsilon :
            newaction = np.argmax(self.Qm[state,:])
        else:
            newaction = np.random.randint(0, 10)
    
        # Set variables for next loop and reset cycle timer
        self.state = newstate
        self.action = newaction
                                
        self.set_action(newaction)

In [77]:
def run(learners = []):
    cycle_timer = 0
    while traci.simulation.getMinExpectedNumber() > 0:
        traci.simulationStep()
        if cycle_timer == 90:
            for learner in learners:
                learner.update()
            cycle_timer = 0
        else:
            cycle_timer = cycle_timer + 1    
        combined_queue.append(traci.lane.getLastStepHaltingNumber(learners[0].laneids[0])+
                                 traci.lane.getLastStepHaltingNumber(learners[0].laneids[1])+
                                 traci.lane.getLastStepHaltingNumber(learners[0].laneids[2])+
                                 traci.lane.getLastStepHaltingNumber(learners[0].laneids[3])+
                                 traci.lane.getLastStepHaltingNumber(learners[1].laneids[0])+
                                 traci.lane.getLastStepHaltingNumber(learners[1].laneids[1])+
                                 traci.lane.getLastStepHaltingNumber(learners[1].laneids[2])+
                                 traci.lane.getLastStepHaltingNumber(learners[1].laneids[3])+
                                 traci.lane.getLastStepHaltingNumber(learners[2].laneids[0])+
                                 traci.lane.getLastStepHaltingNumber(learners[2].laneids[1])+
                                 traci.lane.getLastStepHaltingNumber(learners[2].laneids[2])+
                                 traci.lane.getLastStepHaltingNumber(learners[2].laneids[3]))
        
        if color_vehicles == True:
            vehicleIDs = traci.vehicle.getIDList()
            waiting_time = list()
            for vehicle in vehicleIDs:
                wait = traci.vehicle.getWaitingTime(vehicle)
                if wait >= 20:
                    traci.vehicle.setColor(vehicle,(255,0,0,255))
                elif wait >= 1:
                    traci.vehicle.setColor(vehicle,(255,255,0,255))
                if persistent_color == False:
                    if wait < 1:
                        traci.vehicle.setColor(vehicle,(0,255,0,255))

    
    sys.stdout.flush()

### ORIGINAL SIMPLE CONTROLLER BELOW ###
# The program looks like this
#    <tlLogic id="0" type="static" programID="0" offset="0">
# the locations of the tls are      NESW
#        <phase duration="31" state="GrGr"/>
#        <phase duration="6"  state="yryr"/>
#        <phase duration="31" state="rGrG"/>
#        <phase duration="6"  state="ryry"/>
#    </tlLogic>
#
#def run():
#    """execute the TraCI control loop"""
#    step = 0
#    # we start with phase 2 where EW has green
#    traci.trafficlight.setPhase("0", 2)
#    while traci.simulation.getMinExpectedNumber() > 0:
#        traci.simulationStep()
#        if traci.trafficlight.getPhase("0") == 2:
#            # we are not already switching
#            if traci.inductionloop.getLastStepVehicleNumber("0") > 0:
#                # there is a vehicle from the north, switch
#                traci.trafficlight.setPhase("0", 3)
#            else:
#                # otherwise try to keep green for EW
#                traci.trafficlight.setPhase("0", 2)
#        step += 1
#    traci.close()
#    sys.stdout.flush()

In [10]:
def get_options():
    optParser = optparse.OptionParser()
    optParser.add_option("--nogui", action="store_true",
                         default=False, help="run the commandline version of sumo")
    options, args = optParser.parse_args()
    return options

In [11]:
# Deployment of Network Parser (input is network file)
npa = NetworkParser(os.path.join(networks_dir, used_map, (used_map + '.net.xml')))

In [12]:
nodes = npa.junctions
phase_list = [len(npa.get_phases(k)) for k in npa.get_junction_trafficlights().keys()]
node_list = list(nodes.keys())

In [13]:
# Checks
print(nodes)
print(nodes.keys())
print(phase_list)

{'Centre1': ['iN1_0', 'oCW_0', 'iS1_0', 'iW_0'], 'Centre2': ['iN2_0', 'iCE_0', 'iS2_0', 'iCW_0'], 'Centre3': ['iN3_0', 'iE_0', 'iS3_0', 'oCE_0']}
dict_keys(['Centre1', 'Centre2', 'Centre3'])
[44, 44, 44]


In [14]:
generate_routefile(simulation_length)

In [15]:
sumoBinary = checkBinary('sumo-gui')
# OR
#sumoBinary = checkBinary('sumo')

In [78]:
learners=[Qlearner(ID,lan,p) for ID, lan, p in zip(nodes.keys(),nodes.values(),phase_list)]

In [79]:
combined_queue = []

In [80]:
if reloadQm == True:
    for i in range(len(nodes.keys())):
        learners[i].Qm = np.load(networks_dir+used_map+"/"+"Qm_" + str(i)+ demand  + ".npy")
        print("Loaded pre-trained Q-matrix from file: "+networks_dir+used_map+"/"+"Qm_" + str(i)+ demand  + ".npy")
        with open(networks_dir+used_map+"/"+"Reward_" + str(i) + demand  + ".pickle", 'rb') as fp:
            learners[i].historic_reward = pickle.load(fp)
        print("Loaded Reward history to file: "+networks_dir+used_map+"/"+"Reward_" + str(i) + demand  + ".pickle")

Loaded pre-trained Q-matrix from file: /Users/acabrejasegea/sumo-0.32/Data/Networks/triple_straight_random/Qm_0fixed.npy
Loaded Reward history to file: /Users/acabrejasegea/sumo-0.32/Data/Networks/triple_straight_random/Reward_0fixed.pickle
Loaded pre-trained Q-matrix from file: /Users/acabrejasegea/sumo-0.32/Data/Networks/triple_straight_random/Qm_1fixed.npy
Loaded Reward history to file: /Users/acabrejasegea/sumo-0.32/Data/Networks/triple_straight_random/Reward_1fixed.pickle
Loaded pre-trained Q-matrix from file: /Users/acabrejasegea/sumo-0.32/Data/Networks/triple_straight_random/Qm_2fixed.npy
Loaded Reward history to file: /Users/acabrejasegea/sumo-0.32/Data/Networks/triple_straight_random/Reward_2fixed.pickle


In [81]:
traci.start([sumoBinary, "-c", os.path.join(networks_dir, used_map, (used_map + '.sumocfg')),
                             "--tripinfo-output", os.path.join((notebook + '_out'), 'tripinfo.xml')])

Could not connect to TraCI server at localhost:63999 [Errno 61] Connection refused
 Retrying in 1 seconds


(17, 'SUMO 0.32.0')

In [82]:
run(learners)

/Users/acabrejasegea/sumo-0.32/tools/traci/domain.py:112: UserWarning: The domain trafficlights is deprecated, use trafficlight instead.
  self._name, self._deprecatedFor))  # , DeprecationWarning)


FatalTraCIError: connection closed by SUMO

In [74]:
if saveQm == True:
    for i in range(len(nodes.keys())):
        np.save(networks_dir+used_map+"/"+"Qm_" + str(i) + demand + ".npy", learners[i].Qm)
        print("Saved Q-matrix to file: "+networks_dir+used_map+"/"+"Qm_" + str(i) + demand + ".npy")
        with open(networks_dir+used_map+"/"+"Reward_" + str(i) + demand + ".pickle", 'wb') as fp:
            pickle.dump(learners[i].historic_reward, fp)
        print("Saved Reward history to file: "+networks_dir+used_map+"/"+"Reward_" + str(i) + demand + ".pickle")

Saved Q-matrix to file: /Users/acabrejasegea/sumo-0.32/Data/Networks/triple_straight_random/Qm_0fixed.npy
Saved Reward history to file: /Users/acabrejasegea/sumo-0.32/Data/Networks/triple_straight_random/Reward_0fixed.pickle
Saved Q-matrix to file: /Users/acabrejasegea/sumo-0.32/Data/Networks/triple_straight_random/Qm_1fixed.npy
Saved Reward history to file: /Users/acabrejasegea/sumo-0.32/Data/Networks/triple_straight_random/Reward_1fixed.pickle
Saved Q-matrix to file: /Users/acabrejasegea/sumo-0.32/Data/Networks/triple_straight_random/Qm_2fixed.npy
Saved Reward history to file: /Users/acabrejasegea/sumo-0.32/Data/Networks/triple_straight_random/Reward_2fixed.pickle


In [ ]:
learners[1].Qm

In [ ]:
plt.figure(figsize=(20,10))
plot_series = [x+y+z for x,y,z in zip(learners[0].historic_reward,learners[1].historic_reward,learners[2].historic_reward)]
plt.plot(plot_series[0:10000])

xcoords = [0,2500,5000,7500,10000]
for xc in xcoords:
    plt.axvline(x=xc, color='red')
    
texts = ['Demand Regime 1', 'Demand Regime 2', 'Demand Regime 3', 'Demand Regime 4' ]
for i in range(len(texts)):
    plt.text(xcoords[i]+550,-125,texts[i], fontsize = 14, color='red')

plt.title("Instantaneous reward over time, "+r'$\alpha = 0.95, $' + r'$\epsilon=0.025$', fontsize = 20)
plt.xlabel("Time [cycles]"       , fontsize=16)
plt.ylabel("Instantaneous Reward", fontsize=16)

In [ ]:
trained_cycle = combined_queue[0:90000]
#untrained_cycle = combined_queue

In [ ]:
trained_cycle = trained_cycle[0:90000]
untrained_cycle = untrained_cycle[0:90000]

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(trained_cycle[0:15000], color = 'blue', label = "Trained Controller")

 
plt.plot(untrained_cycle[0:15000], color = 'red', label = "Untrained Controller")
plt.legend(prop={'size' : 16})

plt.title("Total stopped vehicles in the model", fontsize = 20)
plt.xlabel("Time [seconds]"       , fontsize=16)
plt.ylabel("Aggregated queue length [vehicles]", fontsize=16)